In [1]:
import os

In [2]:
os.getcwd()

'/home/ubuntu/projects/llm-models/llm-models/notebook'

In [8]:
from models.deepseek.bidirGPT import get_bidirectinoal_gpt_layer_with_transformer_engine_spec

In [4]:
import importlib, os

from models.deepseek import model
importlib.reload(model)

/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/ubuntu/projects/llm-dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'models.deepseek.model' from '/home/ubuntu/projects/llm-models/llm-models/models/deepseek/model.py'>

In [ ]:
import importlib, os

from models.deepseek import model
importlib.reload(model)

In [ ]:
import sys
sys.path.insert(0, "/home/ubuntu/projects/Megatron-LM")
from gpt_builders import gpt_builder

In [ ]:
import torch.distributed as dist

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12356'
dist.init_process_group(backend='nccl', rank=0, world_size=1)

In [ ]:
from functools import partial
from megatron.training.yaml_arguments import load_yaml, core_transformer_config_from_yaml
from megatron.training import get_args
from types import SimpleNamespace

from megatron.training.initialize import initialize_megatron

# Load YAML and build transformer config once
args_from_yaml = load_yaml("/home/ubuntu/projects/llm-models/llm-models/models/bidirectional/gpt_config.yaml")
transformer_config = core_transformer_config_from_yaml(args_from_yaml, "language_model")

transfomer_key = 'language_model'
# 1. Convert the base YAML object to a dictionary
args_dict = vars(args_from_yaml).copy()

# 2. Safely merge the nested dictionaries
# Using .update() ensures that if a key exists in both, the nested one takes precedence
transfomer_key = 'language_model'
if hasattr(args_from_yaml, transfomer_key):
    args_dict.update(vars(getattr(args_from_yaml, transfomer_key)))

if hasattr(args_from_yaml, 'model_parallel'):
    args_dict.update(vars(args_from_yaml.model_parallel))

# 3. Create the namespace from the merged dictionary
args = SimpleNamespace(**args_dict)

initialize_megatron(
    extra_args_provider=None,
    args_defaults=vars(args),
    ignore_unknown_args=True # Useful if your YAML has extra fields
)

# Provide gpt_builder with that config (config=... skips internal config loading)
def model_provider_with_yaml_config(pre_process=True, post_process=True, vp_stage=None):
    # You must still have get_args() returning the full args (e.g. from same YAML)
    # args = get_args()

        # Initalize and get arguments, timers, and Tensorboard writer.
    extra_args_provider, args_defaults, get_embedding_ranks, get_position_embedding_ranks = None, None, None, None
    store = None
    
    return gpt_builder(args, pre_process, post_process, vp_stage, config=transformer_config)

model = model_provider_with_yaml_config()
# Then in pretrain:
# pretrain(..., partial(model_provider, model_provider_with_yaml_config), ...)

In [ ]:
from megatron.core import parallel_state

parallel_state.model_parallel_is_initialized()

In [ ]:
from megatron.core.models.gpt import GPTModel
from megatron.core.transformer.spec_utils import import_module

import megatron.core.models.gpt as gpt
import importlib
importlib.reload(gpt)

In [ ]:
pre_process = True
post_process = True
mtp_block_spec = None
vp_stage = None

model = gpt.GPTModel(
    config=config,
    transformer_layer_spec=transformer_layer_spec,
    vocab_size=args.padded_vocab_size,
    max_sequence_length=args.max_position_embeddings,
    pre_process=pre_process,
    post_process=post_process,
    fp16_lm_cross_entropy=args.fp16_lm_cross_entropy,
    parallel_output=True,
    share_embeddings_and_output_weights=not args.untie_embeddings_and_output_weights,
    position_embedding_type=args.position_embedding_type,
    rotary_percent=args.rotary_percent,
    rotary_base=args.rotary_base,
    rope_scaling=args.use_rope_scaling,
    mtp_block_spec=mtp_block_spec,
    vp_stage=vp_stage,
)

In [ ]:
from megatron.training import get_args

In [ ]:
from functools import partial
from megatron.training.yaml_arguments import load_yaml, core_transformer_config_from_yaml

# Load YAML and build transformer config once
args_from_yaml = load_yaml("/home/ubuntu/projects/llm-models/llm-models/models/bidirectional/gpt_config.yaml")
transformer_config = core_transformer_config_from_yaml(args_from_yaml, "language_model")


In [ ]:
from functools import partial
from megatron.training.yaml_arguments import load_yaml, core_transformer_config_from_yaml

# Load YAML and build transformer config once
args_from_yaml = load_yaml("/home/ubuntu/projects/llm-models/llm-models/models/bidirectional/gpt_config.yaml")
transformer_config = core_transformer_config_from_yaml(args_from_yaml, "language_model")

# Provide gpt_builder with that config (config=... skips internal config loading)
def model_provider_with_yaml_config(pre_process=True, post_process=True, vp_stage=None):
    # You must still have get_args() returning the full args (e.g. from same YAML)
    args = get_args()
    return gpt_builder(args, pre_process, post_process, vp_stage, config=transformer_config)

# Then in pretrain:
pretrain(..., partial(model_provider, model_provider_with_yaml_config), ...)